# Entry Exit Analysis
Simple analysis to identify entry and exit point candles using stock data. Not using index data, not sure if it's reliable since it's commulative data of many underlying stocks.

In [2]:
import datetime as dt
import utils as ut
import pandas as pd
import icharts as ic
from functools import cache
from constants import *


TEST_START = dt.datetime.strptime("2023-01-01", "%Y-%m-%d")
TEST_END = dt.datetime.strptime("2023-12-31", "%Y-%m-%d")
SYMBOL = "HDFCBANK"
IC_SYMBOL = "HDFCBANK"
INTERVAL = ut.INTERVAL_MIN1
EXCHANGE = ut.EXCHANGE_NSE

pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", 200)
# pd.options.display.float_format = lambda x: '%.2f' % x

def build_date_range(date_start, date_end, symbol):
    date_range = []
    cur_date = date_start
    while cur_date < date_end:
        if cur_date.weekday() not in [5, 6]:
            has_data, _ = ut.has_data(symbol, cur_date, interval=INTERVAL, exchange=EXCHANGE)
            if has_data:
                date_range.append(cur_date)
        cur_date += dt.timedelta(days=1)
    return date_range

all_dates = pd.DataFrame({"trade_date": build_date_range(TEST_START, TEST_END, SYMBOL)})
all_dates_shuffled = all_dates.sample(frac=1, random_state=42)

train_size = int(0.5 * len(all_dates_shuffled))
train_dates = all_dates_shuffled.iloc[:train_size]
test_dates = all_dates_shuffled.iloc[train_size:]
# train_dates = all_dates
train_dates = train_dates.sort_values(by="trade_date")
train_dates.set_index("trade_date", inplace=True)
test_dates = test_dates.sort_values(by="trade_date")
test_dates.set_index("trade_date", inplace=True)

def get_intraday_data(date):
    return ut.get_data(symbol=SYMBOL, date=date, interval=INTERVAL, exchange=EXCHANGE)

train_dates = train_dates.copy()

nifty_candles = train_dates.index.to_series().apply(get_intraday_data).to_list()
# nifty_candles = pd.DataFrame(nifty_candles)
nifty_candles = pd.concat(nifty_candles)

In [5]:
first = nifty_candles.loc[nifty_candles.index.date == nifty_candles.iloc[0].name.date()]
first.loc[:, "pc"] = first.close.diff()
first.loc[:, "pcpv"] = 1000 * first.pc / first.volume

/tmp/ipykernel_136410/3646270214.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first.loc[:, "pc"] = first.close.diff()
/tmp/ipykernel_136410/3646270214.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first.loc[:, "pcpv"] = 1000 * first.pc / first.volume


In [14]:
# first.loc[first.pcpv.abs() < 0.01]
# first.loc[first.volume > 20000].sort_values(by='pcpv')
first.sort_values(by='pcpv')

,open,high,low,close,volume,pc,pcpv
date,,,,,,,
2023-01-02 11:59:00+05:30,1630.30,1630.40,1629.55,1629.55,995,-0.75,-0.753769
2023-01-02 09:57:00+05:30,1631.20,1631.60,1630.00,1630.05,2471,-1.85,-0.748685
2023-01-02 11:32:00+05:30,1633.05,1633.05,1631.55,1631.55,2126,-1.50,-0.705550
2023-01-02 11:56:00+05:30,1630.60,1631.00,1630.05,1630.05,1651,-0.95,-0.575409
2023-01-02 14:29:00+05:30,1626.30,1626.80,1625.45,1625.50,2546,-1.30,-0.510605
...,...,...,...,...,...,...,...
2023-01-02 11:48:00+05:30,1631.60,1632.30,1631.60,1632.25,1206,0.65,0.538972
2023-01-02 10:17:00+05:30,1631.40,1632.00,1631.20,1631.95,1296,0.80,0.617284
2023-01-02 11:05:00+05:30,1635.00,1635.85,1635.00,1635.65,946,0.65,0.687104


In [2]:
groups = nifty_candles.groupby(nifty_candles.index.date)

In [3]:
for date, ddate in groups:
    print(date)

2023-01-02
2023-01-04
2023-01-09
2023-01-10
2023-01-13
2023-01-16
2023-01-18
2023-01-23
2023-01-24
2023-01-27
2023-01-30
2023-02-06
2023-02-07
2023-02-08
2023-02-10
2023-02-13
2023-02-14
2023-02-15
2023-02-17
2023-02-21
2023-02-24
2023-03-01
2023-03-02
2023-03-08
2023-03-09
2023-03-16
2023-03-22
2023-03-23
2023-03-29
2023-04-10
2023-04-11
2023-04-12
2023-04-13
2023-04-17
2023-04-21
2023-04-25
2023-04-26
2023-04-27
2023-04-28
2023-05-02
2023-05-05
2023-05-09
2023-05-10
2023-05-11
2023-05-17
2023-05-22
2023-05-24
2023-05-25
2023-05-26
2023-05-31
2023-06-01
2023-06-06
2023-06-12
2023-06-13
2023-06-15
2023-06-16
2023-06-19
2023-06-20
2023-06-21
2023-06-23
2023-06-27
2023-06-28
2023-07-05
2023-07-06
2023-07-07
2023-07-10
2023-07-17
2023-07-24
2023-07-26
2023-07-27
2023-07-31
2023-08-02
2023-08-04
2023-08-07
2023-08-08
2023-08-14
2023-08-16
2023-08-17
2023-08-18
2023-08-21
2023-08-24
2023-08-29
2023-08-31
2023-09-01
2023-09-06
2023-09-08
2023-09-12
2023-09-13
2023-09-15
2023-09-20
2023-09-21